# Partie 4.3 - Dashboard executif

Ce notebook construit un dashboard multi-panneaux (2x3) synthetisant les principaux
indicateurs de consommation energetique des batiments publics regionaux.

**Entree :** `../output/consommations_enrichies.parquet`

**Sortie :** `../output/figures/dashboard_energie.png` (300 dpi)

In [ ]:
# --- Imports et chargement des donnees ---

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

warnings.filterwarnings("ignore")

# Configuration matplotlib pour un rendu professionnel
plt.rcParams.update({
    "font.size": 11,
    "axes.titlesize": 13,
    "axes.labelsize": 11,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "legend.fontsize": 9,
    "figure.facecolor": "white",
    "axes.facecolor": "#FAFAFA",
    "axes.grid": True,
    "grid.alpha": 0.3,
    "grid.linestyle": "--"
})

# Creation du repertoire de sortie
os.makedirs("../output/figures", exist_ok=True)

# --- Chargement des donnees enrichies ---
df = pd.read_parquet("../output/consommations_enrichies.parquet")

# Conversion de la colonne timestamp en datetime si necessaire
if "timestamp" in df.columns:
    df["timestamp"] = pd.to_datetime(df["timestamp"])

print(f"Dimensions du jeu de donnees : {df.shape[0]:,} lignes x {df.shape[1]} colonnes")
print(f"Colonnes : {list(df.columns)}")
print(f"Periode : {df['timestamp'].min()} -> {df['timestamp'].max()}")
print(f"Types d'energie : {sorted(df['type_energie'].unique())}")
print(f"Types de batiment : {sorted(df['type'].unique())}")
print(f"Communes : {sorted(df['commune'].unique())}")
print(f"Classes DPE : {sorted(df['classe_energetique'].unique())}")
df.head()

## Construction du dashboard multi-panneaux (2x3)

Le dashboard comprend 6 panneaux :
1. Evolution de la consommation totale (6 derniers mois)
2. Top 10 des batiments les plus energivores (consommation/m2)
3. Repartition des couts par type d'energie
4. Consommation moyenne par classe DPE
5. Consommation par commune (empilee par type d'energie)
6. Economies potentielles par amelioration de classe DPE

In [ ]:
# =============================================================================
# DASHBOARD EXECUTIF - BATIMENTS PUBLICS REGIONAUX
# =============================================================================

# --- Palette de couleurs coherente ---
COULEURS_ENERGIE = {
    "electricite": "#3498DB",  # bleu
    "gaz": "#E74C3C",         # rouge
    "eau": "#2ECC71"           # vert
}

COULEURS_TYPE_BAT = {
    "ecole": "#3498DB",
    "mairie": "#E74C3C",
    "piscine": "#2ECC71",
    "gymnase": "#F39C12",
    "mediatheque": "#9B59B6"
}

COULEURS_DPE = {
    "A": "#1B5E20",  # vert fonce
    "B": "#388E3C",  # vert
    "C": "#7CB342",  # vert clair
    "D": "#FDD835",  # jaune
    "E": "#FB8C00",  # orange
    "F": "#E53935",  # rouge
    "G": "#B71C1C"   # rouge fonce
}

# --- Determination de la plage de dates ---
date_min = df["timestamp"].min()
date_max = df["timestamp"].max()
sous_titre_dates = f"Periode d'analyse : {date_min.strftime('%d/%m/%Y')} - {date_max.strftime('%d/%m/%Y')}"

# =============================================================================
# Creation de la figure 2x3
# =============================================================================
fig, axes = plt.subplots(2, 3, figsize=(20, 14))

# =============================================================================
# PANNEAU 1 (haut-gauche) : Evolution de la consommation totale (6 derniers mois)
# =============================================================================
ax1 = axes[0, 0]

# Extraction du mois sous forme de periode pour un tri correct
df["mois_period"] = df["timestamp"].dt.to_period("M")

# Identification des 6 derniers mois de donnees
mois_uniques = sorted(df["mois_period"].unique())
derniers_6_mois = mois_uniques[-6:]

df_6mois = df[df["mois_period"].isin(derniers_6_mois)].copy()

# Agregation mensuelle par type d'energie
conso_mensuelle = (
    df_6mois
    .groupby(["mois_period", "type_energie"])["consommation"]
    .sum()
    .reset_index()
)

# Labels des mois en francais
noms_mois_fr = {
    1: "Jan", 2: "Fev", 3: "Mar", 4: "Avr", 5: "Mai", 6: "Jun",
    7: "Jul", 8: "Aou", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"
}

for type_energie in ["electricite", "gaz", "eau"]:
    masque = conso_mensuelle["type_energie"] == type_energie
    donnees = conso_mensuelle[masque].sort_values("mois_period")
    if len(donnees) > 0:
        labels_x = [f"{noms_mois_fr[p.month]} {p.year}" for p in donnees["mois_period"]]
        ax1.plot(
            labels_x,
            donnees["consommation"].values,
            marker="o",
            linewidth=2.5,
            markersize=7,
            label=type_energie.capitalize(),
            color=COULEURS_ENERGIE[type_energie]
        )

ax1.set_title("Evolution de la consommation totale\n(6 derniers mois)", fontweight="bold", pad=10)
ax1.set_xlabel("Mois")
ax1.set_ylabel("Consommation totale")
ax1.legend(loc="upper left", framealpha=0.9)
ax1.tick_params(axis="x", rotation=30)
ax1.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x:,.0f}"))

# =============================================================================
# PANNEAU 2 (haut-centre) : Top 10 batiments les plus energivores
# =============================================================================
ax2 = axes[0, 1]

# Calcul de la consommation totale par m2 par batiment
conso_par_bat = (
    df.groupby(["batiment_id", "nom", "type", "surface_m2"])["consommation_par_m2"]
    .sum()
    .reset_index()
    .rename(columns={"consommation_par_m2": "conso_totale_par_m2"})
    .sort_values("conso_totale_par_m2", ascending=False)
    .head(10)
)

# Tri ascendant pour affichage horizontal (le plus haut en haut)
conso_par_bat = conso_par_bat.sort_values("conso_totale_par_m2", ascending=True)

# Couleurs par type de batiment
couleurs_barres = [COULEURS_TYPE_BAT.get(t, "#95A5A6") for t in conso_par_bat["type"]]

barres = ax2.barh(
    conso_par_bat["nom"],
    conso_par_bat["conso_totale_par_m2"],
    color=couleurs_barres,
    edgecolor="white",
    linewidth=0.5,
    height=0.7
)

# Legende par type de batiment (uniquement les types presents dans le top 10)
types_presents = conso_par_bat["type"].unique()
legendes_type = [
    plt.Rectangle((0, 0), 1, 1, fc=COULEURS_TYPE_BAT.get(t, "#95A5A6"))
    for t in sorted(types_presents)
]
ax2.legend(
    legendes_type,
    [t.capitalize() for t in sorted(types_presents)],
    loc="lower right",
    fontsize=8,
    framealpha=0.9,
    title="Type",
    title_fontsize=9
)

ax2.set_title("Top 10 batiments les plus energivores\n(consommation totale / m2)", fontweight="bold", pad=10)
ax2.set_xlabel("Consommation totale par m2")
ax2.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x:,.0f}"))

# =============================================================================
# PANNEAU 3 (haut-droite) : Repartition des couts par type d'energie
# =============================================================================
ax3 = axes[0, 2]

# Calcul des couts totaux par type d'energie
couts_par_energie = (
    df.groupby("type_energie")["cout"]
    .sum()
    .reindex(["electricite", "gaz", "eau"])
)

# Couleurs correspondantes
couleurs_pie = [COULEURS_ENERGIE[e] for e in couts_par_energie.index]

# Labels avec montants
total_cout = couts_par_energie.sum()
labels_pie = [
    f"{e.capitalize()}\n{montant:,.0f} EUR"
    for e, montant in zip(couts_par_energie.index, couts_par_energie.values)
]

wedges, texts, autotexts = ax3.pie(
    couts_par_energie.values,
    labels=labels_pie,
    autopct="%1.1f%%",
    colors=couleurs_pie,
    startangle=90,
    pctdistance=0.75,
    wedgeprops={"edgecolor": "white", "linewidth": 2},
    textprops={"fontsize": 10}
)

# Mise en forme des pourcentages
for autotext in autotexts:
    autotext.set_fontweight("bold")
    autotext.set_fontsize(11)

ax3.set_title("Repartition des couts\npar type d'energie", fontweight="bold", pad=10)

# Annotation du cout total au centre
ax3.text(
    0, 0,
    f"Total\n{total_cout:,.0f} EUR",
    ha="center", va="center",
    fontsize=11, fontweight="bold",
    color="#2C3E50"
)

# =============================================================================
# PANNEAU 4 (bas-gauche) : Consommation moyenne par classe DPE
# =============================================================================
ax4 = axes[1, 0]

# Classes DPE de A a G
ordre_dpe = ["A", "B", "C", "D", "E", "F", "G"]
classes_presentes = [c for c in ordre_dpe if c in df["classe_energetique"].unique()]

# Calcul de la moyenne et de l'ecart-type de consommation_par_m2 par classe
stats_dpe = (
    df[df["classe_energetique"].isin(classes_presentes)]
    .groupby("classe_energetique")["consommation_par_m2"]
    .agg(["mean", "std"])
    .reindex(classes_presentes)
)

couleurs_dpe_barres = [COULEURS_DPE[c] for c in classes_presentes]

ax4.bar(
    classes_presentes,
    stats_dpe["mean"],
    yerr=stats_dpe["std"],
    capsize=4,
    color=couleurs_dpe_barres,
    edgecolor="white",
    linewidth=0.8,
    error_kw={"elinewidth": 1.2, "capthick": 1.2, "color": "#2C3E50"},
    width=0.65
)

# Annotations des valeurs sur chaque barre
for i, (classe, row) in enumerate(stats_dpe.iterrows()):
    ax4.text(
        i, row["mean"] + row["std"] + stats_dpe["mean"].max() * 0.02,
        f"{row['mean']:.1f}",
        ha="center", va="bottom",
        fontsize=8, fontweight="bold", color="#2C3E50"
    )

ax4.set_title("Consommation moyenne par classe DPE\n(avec ecart-type)", fontweight="bold", pad=10)
ax4.set_xlabel("Classe energetique (DPE)")
ax4.set_ylabel("Consommation moyenne par m2")

# =============================================================================
# PANNEAU 5 (bas-centre) : Consommation par commune (empilee par energie)
# =============================================================================
ax5 = axes[1, 1]

# Agregation par commune et type d'energie
conso_commune = (
    df.groupby(["commune", "type_energie"])["consommation"]
    .sum()
    .reset_index()
)

# Pivot pour barres empilees
conso_pivot = conso_commune.pivot_table(
    index="commune",
    columns="type_energie",
    values="consommation",
    aggfunc="sum"
).fillna(0)

# Reordonnancement des colonnes
colonnes_ordre = [c for c in ["electricite", "gaz", "eau"] if c in conso_pivot.columns]
conso_pivot = conso_pivot[colonnes_ordre]

# Tri par consommation totale decroissante
conso_pivot["total"] = conso_pivot.sum(axis=1)
conso_pivot = conso_pivot.sort_values("total", ascending=True)
conso_pivot = conso_pivot.drop(columns="total")

# Barres empilees horizontales
gauche = np.zeros(len(conso_pivot))
for col in colonnes_ordre:
    ax5.barh(
        conso_pivot.index,
        conso_pivot[col],
        left=gauche,
        color=COULEURS_ENERGIE[col],
        label=col.capitalize(),
        edgecolor="white",
        linewidth=0.3,
        height=0.7
    )
    gauche += conso_pivot[col].values

ax5.set_title("Consommation par commune\n(par type d'energie)", fontweight="bold", pad=10)
ax5.set_xlabel("Consommation totale")
ax5.legend(loc="lower right", fontsize=8, framealpha=0.9)
ax5.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x:,.0f}"))

# =============================================================================
# PANNEAU 6 (bas-droite) : Economies potentielles par amelioration DPE
# =============================================================================
ax6 = axes[1, 2]

# Strategie : calculer le cout moyen par m2 pour chaque classe DPE,
# puis estimer les economies si chaque batiment etait ameliore d'une classe.

# Cout moyen par m2 et par classe DPE
cout_moyen_dpe = (
    df.groupby("classe_energetique")
    .apply(lambda g: g["cout"].sum() / g["surface_m2"].iloc[0] if g["surface_m2"].iloc[0] > 0 else 0)
)

# Recalcul plus robuste : cout total / surface totale par classe
stats_cout_dpe = (
    df.groupby(["classe_energetique", "batiment_id", "surface_m2"])["cout"]
    .sum()
    .reset_index()
)
cout_par_classe = (
    stats_cout_dpe
    .groupby("classe_energetique")
    .agg(cout_total=("cout", "sum"), surface_totale=("surface_m2", "sum"))
)
cout_par_classe["cout_par_m2"] = cout_par_classe["cout_total"] / cout_par_classe["surface_totale"]

# Mapping classe -> classe amelioree (B->A, C->B, ..., G->F). A reste A.
mapping_amelioration = {"B": "A", "C": "B", "D": "C", "E": "D", "F": "E", "G": "F"}

# Calcul du cout actuel et du cout potentiel par classe
economies_data = []
classes_ameliorables = [c for c in ["B", "C", "D", "E", "F", "G"] if c in cout_par_classe.index]

for classe in classes_ameliorables:
    classe_cible = mapping_amelioration[classe]
    if classe_cible in cout_par_classe.index:
        cout_actuel = cout_par_classe.loc[classe, "cout_total"]
        surface = cout_par_classe.loc[classe, "surface_totale"]
        cout_potentiel = cout_par_classe.loc[classe_cible, "cout_par_m2"] * surface
        economies_data.append({
            "classe": f"{classe} -> {classe_cible}",
            "Cout actuel": cout_actuel,
            "Cout apres amelioration": cout_potentiel,
            "Economie": cout_actuel - cout_potentiel
        })

df_economies = pd.DataFrame(economies_data)

if len(df_economies) > 0:
    # Positions des barres
    x_pos = np.arange(len(df_economies))
    largeur = 0.35

    barres_actuel = ax6.bar(
        x_pos - largeur / 2,
        df_economies["Cout actuel"],
        largeur,
        label="Cout actuel",
        color="#E74C3C",
        edgecolor="white",
        linewidth=0.5
    )

    barres_potentiel = ax6.bar(
        x_pos + largeur / 2,
        df_economies["Cout apres amelioration"],
        largeur,
        label="Cout apres amelioration",
        color="#27AE60",
        edgecolor="white",
        linewidth=0.5
    )

    ax6.set_xticks(x_pos)
    ax6.set_xticklabels(df_economies["classe"], rotation=30, ha="right")
    ax6.legend(loc="upper left", fontsize=8, framealpha=0.9)

    # Annotations des economies
    for i, row in df_economies.iterrows():
        economie_pct = (row["Economie"] / row["Cout actuel"]) * 100 if row["Cout actuel"] > 0 else 0
        hauteur_max = max(row["Cout actuel"], row["Cout apres amelioration"])
        ax6.annotate(
            f"-{economie_pct:.0f}%",
            xy=(i, hauteur_max),
            xytext=(0, 8),
            textcoords="offset points",
            ha="center", va="bottom",
            fontsize=8, fontweight="bold",
            color="#27AE60"
        )

ax6.set_title("Economies potentielles par\namelioration de classe DPE", fontweight="bold", pad=10)
ax6.set_xlabel("Amelioration de classe")
ax6.set_ylabel("Cout total (EUR)")
ax6.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x:,.0f}"))

# =============================================================================
# Titres et mise en forme globale
# =============================================================================
fig.suptitle(
    "Dashboard Energetique - Batiments Publics Regionaux",
    fontsize=20, fontweight="bold", color="#2C3E50",
    y=0.98
)

fig.text(
    0.5, 0.95,
    sous_titre_dates,
    ha="center", va="top",
    fontsize=12, fontstyle="italic", color="#7F8C8D"
)

plt.tight_layout(rect=[0, 0, 1, 0.93])

# --- Sauvegarde ---
chemin_sortie = "../output/figures/dashboard_energie.png"
fig.savefig(chemin_sortie, dpi=300, bbox_inches="tight", facecolor="white")
plt.show()

print(f"\nDashboard sauvegarde : {chemin_sortie}")
print(f"Resolution : 300 dpi")
print(f"Dimensions : 20 x 14 pouces")

## Conclusion

Ce dashboard executif synthetise les principaux indicateurs de performance energetique
des batiments publics regionaux en une figure unique a 6 panneaux.

**Enseignements cles :**

1. **Evolution temporelle** : La consommation de gaz presente un pic marque en hiver tandis que
   la consommation electrique reste plus stable au fil des mois. La consommation d'eau suit un
   profil distinct, moins affecte par la saisonnalite.

2. **Batiments energivores** : Le top 10 des batiments les plus consommateurs par m2 revele
   que les piscines et gymnases dominent ce classement, ce qui s'explique par les besoins
   specifiques en chauffage de l'eau et en ventilation.

3. **Repartition des couts** : L'electricite represente la part la plus importante du budget
   energetique, suivie du gaz. Le cout de l'eau reste proportionnellement modeste mais non
   negligeable.

4. **Classe DPE** : On observe une progression nette de la consommation par m2 entre les
   classes A (les plus performantes) et G (les moins performantes), confirmant la pertinence
   du diagnostic de performance energetique comme indicateur.

5. **Disparites communales** : La consommation totale varie significativement d'une commune
   a l'autre, refletant a la fois le parc immobilier (nombre et taille des batiments) et les
   pratiques energetiques locales.

6. **Potentiel d'economies** : L'amelioration d'une seule classe DPE pour les batiments les
   moins performants (classes E, F, G) pourrait generer des economies substantielles,
   justifiant un programme prioritaire de renovation energetique cible sur ces batiments.

Ce dashboard constitue un outil d'aide a la decision pour les gestionnaires du patrimoine
immobilier public, en identifiant les leviers d'action prioritaires pour reduire la
consommation et les couts energetiques.